# LSTM 

In [73]:
import os
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import log_loss
from matplotlib import pyplot as plt
from wordcloud import WordCloud
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import re
import csv
import numpy as np
import pandas as pd
from string import punctuation
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from spellchecker import SpellChecker
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
import tqdm

%matplotlib inline

In [75]:
path = 'input/'
train = pd.read_csv(path+"train.csv").astype(str)
test = pd.read_csv(path+"test.csv").astype(str)

In [76]:
test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [77]:
train_Q1 = train['question1'].tolist()
train_Q2 = train['question1'].tolist()
test_Q1 = test['question1'].tolist()
test_Q2 = test['question2'].tolist()
labels = np.array(train['is_duplicate'].tolist())
test_ids = np.array(test['test_id'].tolist())

In [92]:
labels = np.array([int(x) for x in labels])

## Pre-process the data
    Use re package separating punctuation from the  word, restore abbreviation 
    
    Remove stop-words
    
    Correct spelling (optional)
    
    Lemmatize word (optional)
    
    Get word stem (optional)

**Spell checker using word2vec**
adapted from 'https://www.kaggle.com/cpmpml/spell-checker-using-word2vec', credit to CPMP

In [101]:
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
words = word2vec.index2word
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank
def words(text): return re.findall(r'\w+', text.lower())
def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)
def correction(word): 
    "Most probable spelling correction for word."
    print(candidates(word))
    return max(candidates(word), key=P)
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

NameError: name 'KeyedVectors' is not defined

In [79]:
def clean(text, remove_stopwords=False, stem_words=False,spell_check=False,lemmer_words=False):

    text = text.lower().split()

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    #Credit to https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    if spell_check:
        text = text.split()
        #spell = SpellChecker()
        #checked_words = [spell.correction(word) for word in text]
        checked_words = [correction(word) for word in text]
        text = " ".join(checked_words)
        
    if lemmer_words:
        text = text.split()
        lemmer = WordNetLemmatizer()
        lemmer_words = [lemmer.lemmatize(word) for word in text]
        text = " ".join(lemmer_words)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    return(text)

In [106]:
SnowballStemmer('english').stem('employer')

'employ'

In [80]:
clean_train_Q1 = [clean(x) for x in train_Q1]
clean_train_Q2 = [clean(x) for x in train_Q2]
clean_test_Q1 = [clean(x) for x in test_Q1]
clean_test_Q2 = [clean(x) for x in test_Q2]

tokenize our sequence

In [81]:
tokenizer = Tokenizer(num_words=200000)
tokenizer.fit_on_texts(clean_train_Q1 + clean_train_Q2 + clean_test_Q1 + clean_test_Q2)

train_seq1 = tokenizer.texts_to_sequences(clean_train_Q1)
train_seq2 = tokenizer.texts_to_sequences(clean_train_Q2)
test_seq1 = tokenizer.texts_to_sequences(clean_test_Q1)
test_seq2 = tokenizer.texts_to_sequences(clean_test_Q2)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))


Found 109652 unique tokens


pad the sequence into the same length where I set to 30 because almost all sentences are range from 3-30

In [82]:
MAX_SEQ = 30
padded_train_seq1 = pad_sequences(train_seq1, maxlen=MAX_SEQ)
padded_train_seq2 = pad_sequences(train_seq2, maxlen=MAX_SEQ)
padded_test_seq1 = pad_sequences(test_seq1, maxlen=MAX_SEQ)
padded_test_seq2 = pad_sequences(test_seq2, maxlen=MAX_SEQ)

## Extract magic features

In [83]:
def safe_divide(x, y, val=0.0):
    if y != 0.0:
        val = float(x) / float(y)
    return val

def get_jaccard(seq1, seq2):
    return 1 -  safe_divide(len(set(seq1) & set(seq2)),len(set(seq1) | set(seq2)))

def get_len(seq):
    return len(seq)

def get_len_unique(seq1,seq2):
    set1 = set(seq1)
    set2 = set(seq2)
    intersection = len(set(seq1) & set(seq2))
    return len(set1) -intersection

def get_features(seq1,seq2):

    jaccard = np.array([ get_jaccard(x1,x2) for x1,x2 in zip(seq1,seq2)]).reshape(-1,1)
    len1 = np.array([ get_len(x1)  for x1 in  seq1]).reshape(-1,1)
    len2 = np.array([ get_len(x2)  for x2 in  seq2]).reshape(-1,1)

    len1_unique = np.array([ get_len_unique(x1,x2)  for x1,x2 in  zip(seq1,seq2)]).reshape(-1,1)
    len2_unique = np.array([ get_len_unique(x2,x1)  for x1,x2 in  zip(seq1,seq2)]).reshape(-1,1)

    len_diff = np.abs(len2-len1)


    features = np.hstack([jaccard,len1,len2,len1_unique,len2_unique,len_diff])
    

    return features
    #return X_jaccard

In [84]:
features = get_features(train['question1'].apply(lambda x:x.split(' ')),train['question2'].apply(lambda x:x.split(' ')))
features_test = get_features(test['question1'].apply(lambda x:x.split(' ')),test['question2'].apply(lambda x:x.split(' ')))

In [85]:
ss = StandardScaler()
ss.fit(np.vstack((features, features_test)))
features = ss.transform(features)
features_test = ss.transform(features_test)

## Get the embedding

In [86]:
def embedding(embedding_file):
    embeddings_index = {}
    f = open(embedding_file)
    count = 0
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %d word vectors of glove.' % len(embeddings_index))
    return embeddings_index
embeddings_index = embedding('glove.6B.300d.txt')
#word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [88]:
em_words = min(200000, len(word_index))+1
embedding_dim = 300
embedding_matrix = np.zeros((em_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    #if word in word2vec.vocab:
    #    embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 37647


## Sample the data

In [93]:
split_rate = 0.1
perm = np.random.permutation(len(padded_train_seq1))
idx_train = perm[:int(len(padded_train_seq1)*(1-split_rate))]
idx_val = perm[int(len(padded_train_seq1)*(1-split_rate)):]

final_train_seq1 = np.vstack((padded_train_seq1[idx_train], padded_train_seq2[idx_train]))
final_train_seq2 = np.vstack((padded_train_seq2[idx_train], padded_train_seq1[idx_train]))
features_train = np.vstack((features[idx_train], features[idx_train]))
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))
final_val_seq1 = np.vstack((padded_train_seq1[idx_val], padded_train_seq2[idx_val]))
final_val_seq2 = np.vstack((padded_train_seq2[idx_val], padded_train_seq1[idx_val]))
features_val = np.vstack((features[idx_val], features[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))

weight_val = np.ones(len(labels_val))
re_weight = True
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

## Model

In [94]:
num_lstm = 200
num_dense = 150
act = 'relu'
embedding_layer = Embedding(em_words,embedding_dim,weights=[embedding_matrix],
                            input_length=MAX_SEQ,trainable=False)
lstm_layer = LSTM(num_lstm, dropout=0.25, recurrent_dropout=0.25)

Q1_input = Input(shape=(MAX_SEQ,), dtype='int32')
embedded_sequences_1 = embedding_layer(Q1_input)
encoding1 = lstm_layer(embedded_sequences_1)

Q2_input = Input(shape=(MAX_SEQ,), dtype='int32')
embedded_sequences_2 = embedding_layer(Q2_input)
encoding2 = lstm_layer(embedded_sequences_2)

features_input = Input(shape=(features.shape[1],))
features_dense = Dense(int(num_dense/2), activation=act)(features_input)

merged = concatenate([encoding1, encoding2, features_dense])
merged = BatchNormalization()(merged)
merged = Dropout(0.25)(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.25)(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [95]:
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

model = Model(inputs=[Q1_input, Q2_input, features_input], outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 30, 300)      32895900    input_29[0][0]                   
                                                                 input_30[0][0]                   
__________________________________________________________________________________________________
input_31 (InputLayer)           (None, 6)            0                                      

In [96]:
early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = 'best_model' + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

In [97]:
class_weight

{0: 1.309028344, 1: 0.472001959}

In [98]:
hist = model.fit([final_train_seq1 , final_train_seq2, features_train], labels_train, \
        validation_data=([final_val_seq1, final_val_seq2, features_val], labels_val, weight_val), \
        epochs=5, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

Train on 727722 samples, validate on 80858 samples
Epoch 1/5
727722/727722 [==============================] - 1711s 2ms/step - loss: 0.3797 - acc: 0.7067 - val_loss: 0.3344 - val_acc: 0.6655
Epoch 2/5
727722/727722 [==============================] - 1575s 2ms/step - loss: 0.3185 - acc: 0.7335 - val_loss: 0.3065 - val_acc: 0.7350
Epoch 3/5
727722/727722 [==============================] - 3159s 4ms/step - loss: 0.3022 - acc: 0.7526 - val_loss: 0.2983 - val_acc: 0.7569
Epoch 4/5
727722/727722 [==============================] - 1419s 2ms/step - loss: 0.2894 - acc: 0.7669 - val_loss: 0.2931 - val_acc: 0.7574
Epoch 5/5
727722/727722 [==============================] - 1429s 2ms/step - loss: 0.2780 - acc: 0.7796 - val_loss: 0.2888 - val_acc: 0.7705


In [100]:
preds = model.predict([padded_test_seq1, padded_test_seq2, features_test], batch_size=8192, verbose=1)
preds += model.predict([padded_test_seq2, padded_test_seq1, features_test], batch_size=8192, verbose=1)
preds /= 2

submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})
submission.to_csv('submission/submission_lstm.csv', index=False)

2345796/2345796 [==============================] - 1854s 791us/step
